In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=8
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A2_019', 'Input_A6_016', 'Input_A1_016',
       'Input_A4_018', 'Input_A4_016', 'Input_A5_016', 'Input_A6_017'],
      dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,8)

train_rmse: 0.008505921572252061
test_rmse: 0.009249846146396823
test_r2: 0.5436609914464399


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-05)

test_rmse_ave: 0.009115614264934825
[0.009405237342328442, 0.007875143159502693, 0.009800286932834593, 0.009865856921933872, 0.007521914929789588, 0.007974452358010926, 0.009107450808243583, 0.009888716055640541, 0.01010274105743251, 0.009614343083631483]


test_r2_ave: 0.5456515961610326
[0.5451934467112469, 0.6457927955311251, 0.489536486155844, 0.48245190163035556, 0.6699661954808496, 0.6419572016995084, 0.5342011225180354, 0.43745468391050724, 0.4655375502695903, 0.5444245777032639]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.025],0.01)

test_rmse_ave: 0.009107383557525316
[0.009369592396116611, 0.007933388090389815, 0.009718423860535301, 0.009875318188896661, 0.007563815235390695, 0.007974343221045635, 0.009073351001855794, 0.009847562073597053, 0.010080229088624803, 0.009637812418800787]


test_r2_ave: 0.5466826244967145
[0.548634260466682, 0.6405339534212691, 0.4980288050150513, 0.48145877783177415, 0.6662790937798011, 0.6419670018551317, 0.5376826490963671, 0.4421272431910037, 0.4679167851085906, 0.5421976752014745]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,8,model)

train_rmse: 0.008763569204631106
test_rmse: 0.00955710219207867
test_r2: 0.5131310213629848


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 6, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.009594181753428889
[0.009576742820043815, 0.008234342727512917, 0.010343216552456923, 0.01008971077463508, 0.008743945685291465, 0.008308080533092684, 0.00906382909449514, 0.010520708709442201, 0.010856770512427484, 0.01020447012489119]


test_r2_ave: 0.4968166163077624
[0.5284553256161528, 0.6127438230159745, 0.4314111246846385, 0.4586993783798716, 0.5540188597801703, 0.6113715392511025, 0.5386524855108219, 0.3632518496415299, 0.38277988623707604, 0.486781890960287]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 13,
            min_samples_leaf =3,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.009281953638784288
[0.00919788220044316, 0.0076671390524333015, 0.009223367614186576, 0.010113928440014321, 0.008533146233002335, 0.00826455322547545, 0.009207040273129932, 0.010028571290706478, 0.010237058213106089, 0.010346849845345226]


test_r2_ave: 0.5292924845334115
[0.5650264178946611, 0.6642568544048165, 0.547867047547295, 0.4560977637261481, 0.575263124918487, 0.6154330400667123, 0.5239584562350499, 0.4214301053719479, 0.4512315936565915, 0.47236044151240597]


In [12]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.025], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 6, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 13,
            min_samples_leaf =3,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.009089551665341993
[0.009364542591251602, 0.00772710804936289, 0.009654189110065147, 0.009805534973212746, 0.007669552235925196, 0.007986283668014042, 0.008960212981211094, 0.009934801789731764, 0.010088301716170249, 0.009704989538475197]


test_r2_ave: 0.5483483336932716
[0.5491206625611642, 0.6589842436205813, 0.5046425183496549, 0.4887613522403542, 0.6568834942789451, 0.6408939919364617, 0.5491402821897755, 0.4321990520679232, 0.4670642192587129, 0.5357935204291429]


In [16]:

stack_mod = StackingRegressor(regressors=[lasso_mod,elastic_mod,lr, random_mod ], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.009157196410304056
[0.009144575538437906, 0.007503654459025851, 0.00922747017313495, 0.010002571115719268, 0.008062578353887042, 0.008183687664156987, 0.009049967391911657, 0.010008417766281042, 0.01032740426023436, 0.010061637380251492]


test_r2_ave: 0.5414054543158593
[0.5700536184384498, 0.6784221487826019, 0.547464740125911, 0.4680088752378525, 0.6208164506229654, 0.6229219096997556, 0.5400625235522569, 0.42375316932820706, 0.44150265931670185, 0.5010484480538919]


In [17]:
ave(skew_data[title],y,0.2,stack_mod,0.6,vote_mod,0.2,elastic_mod)

test_rmse_ave: 0.00906631362535754
[0.009288760372872067, 0.007688996771183956, 0.009556592751206993, 0.00981503449845183, 0.007677295687513264, 0.007982386104020528, 0.008942136064132602, 0.009898424442849954, 0.010103425640894887, 0.009710083920449301]


test_r2_ave: 0.5506619055038191
[0.5563885844827823, 0.6623398317125873, 0.5146072539359886, 0.48777030444291736, 0.6561902993641511, 0.6412444170329525, 0.5509576346175709, 0.4363495680131664, 0.46546511516533184, 0.5353060462707431]
